In [47]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

In [30]:
df = pd.read_csv('/kaggle/input/historical-dataset/historical_data/ADANIENT.csv')
df

,timestamp,open,high,low,close,volume,oi
0,2025-03-10T15:15:00+05:30,2229.80,2230.55,2220.00,2224.40,98264,0
1,2025-03-10T14:45:00+05:30,2225.40,2234.00,2224.90,2230.35,89785,0
2,2025-03-10T14:15:00+05:30,2247.70,2249.05,2224.00,2226.00,53973,0
3,2025-03-10T13:45:00+05:30,2250.95,2252.00,2245.60,2248.70,33624,0
4,2025-03-10T13:15:00+05:30,2261.00,2261.15,2250.95,2250.95,32525,0
...,...,...,...,...,...,...,...
632,2025-01-01T11:15:00+05:30,2545.15,2547.85,2532.10,2537.00,64807,0
633,2025-01-01T10:45:00+05:30,2541.20,2552.75,2539.70,2545.15,96430,0
634,2025-01-01T10:15:00+05:30,2530.80,2545.00,2530.00,2540.00,95404,0
635,2025-01-01T09:45:00+05:30,2554.15,2555.00,2518.25,2531.00,450529,0


In [32]:
# Hyperparameters
SEQ_LEN = 30  # Number of past days used for prediction
PRED_LEN = 1  # Predict the next day's price
D_MODEL = 64  # Transformer model dimension
NHEAD = 4  # Multi-head attention heads
NUM_LAYERS = 3  # Transformer layers
BATCH_SIZE = 32
EPOCHS = 20
LR = 0.001

In [33]:
# Load dataset
data_dir = "/kaggle/working/dataset/splits"

# Function to load and preprocess stock data
def load_stock_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['timestamp'])
    df = df.sort_values(by='timestamp')
    df['Return'] = df['close'].pct_change()  # Use returns instead of raw prices
    df = df.dropna()
    return df[['Return']].values  # Return as NumPy array

# Prepare dataset for Transformer
class StockDataset(Dataset):
    def __init__(self, data, seq_len=SEQ_LEN, pred_len=PRED_LEN):
        self.data = data
        self.seq_len = seq_len
        self.pred_len = pred_len

    def __len__(self):
        return len(self.data) - self.seq_len - self.pred_len + 1

    def __getitem__(self, idx):
        x = self.data[idx:idx + self.seq_len]
        y = self.data[idx + self.seq_len:idx + self.seq_len + self.pred_len]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [34]:
data_dir = "/kaggle/input/historical-dataset/historical_data"  # Path to your dataset folder
all_data = []

# Load and process each CSV file
for file in os.listdir(data_dir):
    if file.endswith(".csv"):
        file_path = os.path.join(data_dir, file)
        stock_name = file.replace(".csv", "")  # Extract stock ticker
        
        # Load data
        df = pd.read_csv(file_path)
        
        # Add stock name as a new column
        df["Stock"] = stock_name
        
        # Append to list
        all_data.append(df)

# Combine all stock data into a single DataFrame
final_dataset = pd.concat(all_data, ignore_index=True)
final_dataset["timestamp"] = pd.to_datetime(final_dataset["timestamp"])  # Convert to datetime format
final_dataset = final_dataset.sort_values(by=["timestamp"])  # Sort by date

In [35]:
final_dataset = final_dataset.drop(columns=["oi"])

In [36]:
print(final_dataset.head())

                      timestamp     open     high      low    close  volume  \
31211 2025-01-01 09:15:00+05:30   308.70   309.15   307.05   307.25  325458   
17198 2025-01-01 09:15:00+05:30  2280.00  2305.00  2280.00  2299.00  108780   
2547  2025-01-01 09:15:00+05:30   292.30   293.65   291.50   293.15  422483   
11465 2025-01-01 09:15:00+05:30   601.50   603.75   596.50   599.55  572072   
5732  2025-01-01 09:15:00+05:30  1214.85  1217.90  1213.50  1216.20  423841   

            Stock  
31211   POWERGRID  
17198  ASIANPAINT  
2547         BPCL  
11465    HINDALCO  
5732     RELIANCE  


In [37]:
# 🟢 Step 2: Encode Categorical Feature (Stock Name)
label_encoder = LabelEncoder()
final_dataset["Stock"] = label_encoder.fit_transform(final_dataset["Stock"])

In [38]:

# 🟢 Step 3: Normalize Features
scaler = MinMaxScaler()
features = ["open", "high", "low", "close", "volume"]  # Feature columns
final_dataset[features] = scaler.fit_transform(final_dataset[features])

In [41]:
# 🟢 Step 4: Create Sequences for Transformer Input
def create_sequences(data, seq_length=30):
    """Converts time series data into sequences for Transformer."""
    sequences, targets = [], []
    
    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i+seq_length]  # Get sequence window
        target = data.iloc[i+seq_length]["close"]  # Predict next close price
        
        sequences.append(seq[features].values)  # Extract numerical features
        targets.append(target)  # Store target
    
    return np.array(sequences), np.array(targets)

In [42]:
# Create sequences
seq_length = 30  # Number of past days to look at
X, y = create_sequences(final_dataset, seq_length)

# 🟢 Step 5: Train-Test Split
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Print dataset shapes
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (24945, 30, 5)
X_test shape: (6237, 30, 5)
y_train shape: (24945,)
y_test shape: (6237,)


In [48]:
# 🟢 Convert NumPy Arrays to PyTorch Tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Make y 2D
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# 🟢 Create PyTorch DataLoader
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [49]:
# 🟢 Define Transformer Model
class StockTransformer(nn.Module):
    def __init__(self, input_dim, embed_dim, num_heads, ff_dim, num_layers, dropout=0.1):
        super(StockTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, embed_dim)  # Feature embedding
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, nhead=num_heads, dim_feedforward=ff_dim, dropout=dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embed_dim, 1)  # Output layer (predict next Close price)

    def forward(self, x):
        x = self.embedding(x)  # Linear projection
        x = x.permute(1, 0, 2)  # PyTorch Transformer expects (seq_len, batch, feature)
        x = self.transformer_encoder(x)
        x = x.mean(dim=0)  # Global average pooling
        x = self.fc(x)  # Regression output
        return x

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [51]:
# 🟢 Initialize Model, Loss, Optimizer
input_dim = X_train.shape[2]  # Number of features
embed_dim = 64
num_heads = 4
ff_dim = 128
num_layers = 3
dropout = 0.1

In [52]:
model = StockTransformer(input_dim, embed_dim, num_heads, ff_dim, num_layers, dropout).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 🟢 Train the Model
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        predictions = model(X_batch)
        loss = criterion(predictions, y_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.6f}")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1/50, Loss: 0.047723
Epoch 2/50, Loss: 0.045033
Epoch 3/50, Loss: 0.044391
Epoch 4/50, Loss: 0.044286
Epoch 5/50, Loss: 0.044068
Epoch 6/50, Loss: 0.044070
Epoch 7/50, Loss: 0.043886
Epoch 8/50, Loss: 0.044003
Epoch 9/50, Loss: 0.043892
Epoch 10/50, Loss: 0.043805
Epoch 11/50, Loss: 0.043883


KeyboardInterrupt: 

In [ ]:
# 🟢 Evaluate the Model
model.eval()
with torch.no_grad():
    total_loss = 0
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        predictions = model(X_batch)
        loss = criterion(predictions, y_batch)
        total_loss += loss.item()

print(f"Test Loss: {total_loss / len(test_loader):.6f}")